# 18-Aspen Hysys
In this exercise we'll discuss how simulators like Aspen Hysys can be used to estimate unit operation behavior, specifically a flash drum. This is an exercise to give you an introduction on vapor liquid equilibrium.

## Aspen Hysys Software Access
You'll log onto one of the computers in the UO lab and open Aspen Hysys. You can also access that software remotely using 'labconnect' for example.

## Aspen Help Video
Link to video that explains a simple flash point calculation or flash drum separation step: [Hysys flash drum video](https://www.youtube.com/watch?v=qijWOzm7irc)

## Vapor Liquid Equilibrium Background
The vapor-liquid equilibrium (VLE) of a component or mixture is important in chemical engineering. You'll take thermodynamics and there spend a significant amount of time on understanding VLE. Simply put, VLE is the ratio of the vapor mole fraction to the liquid mole fraction at a given temperature and pressure.

$$K_i = \dfrac{y_i}{x_i} = \gamma_i\cdot P_i^{sat}/P $$
or for ideal solutions
$$ \dfrac{y_i}{x_i} = P_i^{sat}/P$$

where $K$ is the VLE ratio for a given component $i$, $y_i$ is the vapor mole fraction, $x_i$ is the liquid mole fraction, $P_i^{sat}$ is the saturation pressure of the component, and $P$ is the total pressure.  An ideal solution is one where the vapor and liquid phases are in equilibrium and the saturation pressure is independent of the composition of the liquid phase. Raoult's law is an example of an ideal solution.

## VLE Example Calculation with Python

<img src="https://github.com/clint-bg/comptools/blob/main/lectures/supportfiles/flashingunitbasic.png?raw=true" width="200">

The Feed is at the left and the liquid at the bottom and the vapor at the top. The vapor is richer in the more volatile component (methanol) and the liquid is richer in the less volatile component (ethanol). 

Given the feed compositions, the total feed, the feed vapor fraction, and the pressure there are five variables to solve for:
- The vapor and liquid compositions
- The temperature
- The vapor and liquid flow rates

The variables are found from solving the following equations assuming an ideal solution for binary system:
$$\begin{align*}
F &= L + V \text{ : Mass Balance} \\
z_1\cdot F &= x_1\cdot L + y_1\cdot V/L\cdot L \text{ : Component Balance} \\
H_F &= H_l + H_v \text{ : Energy Balance} \\
P & = x_1\cdot P_1^{sat} + (1-x_1)\cdot P_2^{sat} \text{ : }\Sigma x = 1 \text{, Ideal} \\
y_1 + (1-x_1)\cdot P_2^{sat}/P &= 1 \text{ : } \Sigma y = 1 \text{, Ideal}
\end{align*}$$

In [1]:
## Example VLE calculation
# first import packages
import numpy as np
import param
from scipy.optimize import fsolve #import fsolve from scipy.optimize
import scipy.integrate as integrate #import integrate from scipy to get enthalpy

# define constants
TFlow = 30 #kg/hr, total flow
VoverL = 0.1 # ratio of the vapor flow rate to the liquid flow rate in the feed
P = 1.6e5 # Pa, pressure of the system
z = np.array([0.95,0.05]) # feed composition 
#Tref = 298 # K, reference temperature for heat capacity

#define the properties of the components
class component(param.Parameterized):
    Tref = param.Number(298,doc= "K, reference temperature for heat capacity")
    VPcoef = param.List([1,1,1,1,1],doc="Vapor pressure coefficients to yield Psat in Pa, T in K")
    LHCcoef = param.List([1,1,1,1,1],doc="Liquid heat capacity coefficients to yield Cp in J/kmol/K, T in K")
    IGHCcoeff = param.List([1,1,1,1,1],doc="Ideal gas heat capacity coefficients to yield Cp in J/kmol/K, T in K")
    MW = param.Number(1,doc="Molecular weight in kg/kmol")
    Vap = param.Number(1,doc="Heat of Vaporization in J/kmol at 298K(Tref)")

    def Psat(self,T):
        C = self.VPcoef
        return np.exp(C[0]+C[1]/T+ C[2]*np.log(T)+C[3]*T**C[4])
    def Cpig(self,T):
        C = self.IGHCcoeff
        return C[0]+C[1]*(C[2]/T/np.sinh(C[2]/T))**2+C[3]*(C[4]/T/np.cosh(C[4]/T))**2
    def Cpl(self,T):
        C = self.LHCcoef
        return C[0]+C[1]*T+C[2]*T**2+C[3]*T**3+C[4]*T**4
    def Hl(self,T):
        return integrate.quad(self.Cpl,self.Tref,T)[0]
    def Hv(self,T):
        return self.Vap+integrate.quad(self.Cpl,self.Tref,T)[0]

#vapor pressure coefficient estimates  
methanol = component(VPcoef=[82.7,-6900,-8.86,7.46E-06,2],\
                LHCcoef=[256000, -2740, 14.8, -0.035, 3.27E-05],\
                    IGHCcoeff=[39250, 87900, 1916, 53650, 896],MW=32.04,Vap=3.75e7)
ethanol = component(VPcoef=[73.3,-7122,-7.14,2.88E-06,2],\
               LHCcoef=[102640,-139.63,-0.030341,0.0020386,0.0],\
                IGHCcoeff=[49200,145770,1660,93900,744],MW=46.07,Vap=4.26e7)

In [3]:
# define the function to be solved
def VLE(xvar):
    x = xvar[0] # first element of xvar is the liquid composition of methanol
    y = xvar[1] # next elements of xvar is the vapor composition of methanol
    T = xvar[2] # third element of xvar is the temperature
    L = xvar[3]
    VoverLh = xvar[4]
    V = TFlow*(1 - 1/(1+VoverLh))
    eq = np.zeros(5) # initialize the array of equations
    eq[0] = L*VoverLh + L - TFlow # mass balance
    eq[1] = x*methanol.Psat(T) + (1-x)*ethanol.Psat(T) - P # Raoult's law
    eq[2] = x*L + y*VoverLh*L - TFlow*z[0] # component balance
    eq[3] = (1-x)*ethanol.Psat(T)/P + y - 1 # sum of mole fractions
    #Ethalpy of the Feed
    Vf = TFlow*(1 - 1/(1+VoverL)); Lf = TFlow/(1+VoverL)
    HF = z[0]*(methanol.Hl(T)*Lf+methanol.Hv(T)*Vf)/methanol.MW + z[1]*(ethanol.Hl(T)*Lf+ethanol.Hv(T)*Vf)/ethanol.MW #kJ/hr
    #Ethalpy of the Vapor
    HV = y*methanol.Hv(T)*V/methanol.MW + (1-y)*ethanol.Hv(T)*V/ethanol.MW #kJ/hr
    #Ethanlpy of the Liquid
    HL = x*methanol.Hl(T)*L/methanol.MW + (1-x)*ethanol.Hl(T)*L/ethanol.MW #kJ/hr
    eq[4] = HF - HV - HL
    return eq

# initial guess
xvar0 = np.array([z[0],z[0],350,TFlow/(1+VoverL),VoverL])

# solve the function
out = fsolve(VLE,xvar0)

# print the results
print('The liquid and vapor compositions of methanol are ',out[0:2])
print(f'The temperature is {out[2]-273:.2f} C')
print(f'The vapor and liquid flow rates are {out[3]:.2f} and {TFlow - out[3]:.2f} kg/hr')
print(f'The vapor to liquid ratio is {out[4]:.5f}')

The liquid and vapor compositions of methanol are  [0.94815887 0.96849078]
The temperature is 77.16 C
The vapor and liquid flow rates are 27.28 and 2.72 kg/hr
The vapor to liquid ratio is 0.09957


In [5]:
#How well did the answer satisfy the equations?
VLE(out)

array([ 1.06581410e-14,  4.04543243e-08, -1.24344979e-13,  8.59312621e-14,
        3.16649675e-08])

## VLE Example Calculation with Aspen Plus

### Aspen Plus Steps
1. Open Aspen Plus
2. Add a component to the simulation (I added methanol and ethanol)
3. Specify fluid properties (I used the an Ideal Solution)
4. Add a separator to the simulation
5. Specify the inlet feed composition, feed vapor fraction, and pressure (the same as specified above for the manual calculation)
6. Name the liquid and vapor exit streams
6. Press the play button to run the simulation

<img src="https://raw.githubusercontent.com/clint-bg/comptools/main/lectures/supportfiles/AspenPlusImg.png" width="800">

Compared to the above manual results, the ideal solutions agree fairly well despite differences in the vapor pressure and enthalpies. If you specify a different fluid package that includes non-idealities in the solution, you'll get a slightly different answer (hopefully a more accurate answer).

You should look to do this in your future work as an engineer: Check your answer. Never use just one source. Or in other words, doubt that you got the right answer until you prove to yourself otherwise.

### Aspen HySyS Result for a Flash Drum
The same simulation can be run in Aspen Hysys. The below image shows the results for an Antione Fluid package (ideal solution). The results are similar to Aspen Plus.

Aspen Plus has more compounds.

<img src="https://raw.githubusercontent.com/clint-bg/comptools/main/lectures/supportfiles/AspenHysysImg.png" width="800">

### Additional Simulation Options
You can add complexity by adding a pressure drop through the flash drum (like when the feed may be forced through an orifice). You can also add a heat source to evaporate more material such that the vapor flow rate is increased.